## River Crossing
### Introduction
We looked at the [Wolf, goat and cabbage problem](https://en.wikipedia.org/wiki/Wolf,_goat_and_cabbage_problem) river crossing problem in the unit material. This problem is very simple to solve by hand, and the solution path is not very long, so for this activity we will use another famous river crossing puzzle.

The [missionaries and cannibals](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem) is another toy river crossing problem, and is well-known in the AI literature because it was famously used by [Saul Amarel (1968)](https://web.archive.org/web/20080308224227/http://www.cc.gatech.edu/~jimmyd/summaries/amarel1968-1.html) as an example of problem representation in AI. Versions of the game are known to be at least [1000 years old](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem#History). The problem is also subject of Exercise 3.9 in Russell & Norvig (2016, 3rd ed.) where it is stated as follows (p. 115).

> "Three missionaries and three cannibals are on one side of a river, along with a boat that can hold one or two people. Find a way to get everyone to the other side without ever leaving a group of missionaries in one place outnumbered by the cannibals in that place."

#### Note On Problematic Theme
Before we move on, an important note. In the modern age, the theme of this problem is problematic. The concept inescapably evokes images of colonialism, and was conceived in a time when sadly it was not uncommon to make associations between Black people and “cannibals”. As we said in the unit material, these problems are often repeated with different themes but the same underlying rules. As it happens, this problem was previously more commonly known as the *“jealous husbands”* problem – the puzzle states that no woman can be left unsupervised without her husband also present. For hopefully obvious reasons, I do not find this much of an improvement. 

The extremely short history of AI is dominated by white male voices, and still is today. We will revisit this topic in week 8 when we talk about AI ethics, because as AI continues to spread at a rapid pace this is having a demonstrable impact on people's lives.

Today this puzzle variant is called “missionaries and cannibals” or “jealous husbands” in the AI textbooks; I hope in a number of years we'll have moved beyond these themes entirely. I think it is completely possible (and reasonable) to come up with toy puzzles that feature no element of inequality or violence at all.

For this activity we will stay in line with the textbook and use this common theme, but if you decide to swap out the “missionaries” and the “cannibals” for something else, or even just use abstract labels, then it will not affect the lesson.

#### Before You Start
If you have not already, then take a moment to try to solve this puzzle yourself before we move onto the search based solution.

### Search Based Solution
As you saw in the Tower of Hanoi example, it is possible to solve this kind of problem through uninformed search. The following diagram shows the complete search space of the missionaries and cannibals problem. The initial state is shown on the left and the goal state is all the way to the right. Missionaries are represented by black triangles and cannibals by red circles. Arrows represent state transitions and are labelled with actions, e.g., 2c represents the action of two cannibals crossing the river.

<br><br>
<center>
<figure>
<img src="resources/mc-search-space.png" width=600>
<figcaption>The complete search space of the missionaries and cannibals problem. Credit: <a href=http://www.aiai.ed.ac.uk/~gwickler/missionaries.html>Gerhard Wickler</a></figcaption>
</figure>
</center>
<br><br>

### Your Task
Your task is to write a Python program that solves the missionaries and cannibals problem using **breadth-first search**. The pseudo-code from Russel and Norvig (p. 82) is repeated again below.

<img src="resources/Breadth_first_search.png" width=60%>

Unlike the Towers of Hanoi puzzle, in this task you will have to write all of the supporting code from scratch, with some suggestions. You may wish to use the accompanying "Infrastructure for search algorithms" shown in Section 3.3.1 of the same book (see the unit reading list).

Specifically, you may define a `Node` class with attributes
 * `state`: the state in the state space to which the node corresponds;
 * `parent` (optional): the node in the search tree that generated this node;
 * `action` (optional): the action that was applied to the parent to generate the node;
 
and methods

 * `is_goal_state()`: check whether the Node is the goal state;
 * `get_child_node()`: given an action, return the resulting child state;
 * `is_valid_state()`: would the state result in missionaries getting eaten?
 
Once you have a functioning `Node` class you need to come up with data structures for your `frontier` and your set of  `explored` nodes. 

The next choices you have to make is how to represent states and actions. You may follow Saul Amarel's approach of  representing the current state by a simple vector $<a,b,c>$. The vector's elements $a,b,$ and $c$ represent the number of missionaries on the wrong side, the number of cannibals on the wrong side, and the number of boats on the wrong side, respectively. Since all missionaries, all cannibals, and the boat start on the wrong side, the vector is initialised to $<3,3,1>$. Actions are represented using vector subtraction/addition to manipulate the state vector. For instance, if one cannibal crossed the river, the vector $<0,1,1>$ would be subtracted from the state to yield $<3,2,0>$.

You could also use a single Python tuple to represent the state, at the cost of having to more manually implement the state manipulations.

So you can (but do not have to) use the following structure and define two classes `Node` and `Game`:

In [ ]:
# I recommend you to start coding in another cell below and to keep this cell as a reference.
# That way you can incrementally build --> debug --> build --> debug --> ... your classes  
# instead of trying to do it all at once.

# class Node:
#     def __init__(self, m_wrong_side, c_wrong_side, boat_wrong_side):
#         self.state = ...
    
#     def is_goal_state(self):
#         ...

#     def get_child_node(self, action):
#         ...

#     ...

        
# class Game:
#     def __init__(self):
#         self.initial_node = Node(m_wrong_side=3, c_wrong_side=3, boat_wrong_side=1)
#         ...
    
#     def breadth_first_search(self):
#         ...

In [3]:
import numpy as np

class RiverState:
    def __init__(self, foxes = 3, sheep = 3, boats = 1, state = None, parent = None, action = None):
        # each number represents how many of that item there are in total
        self.foxes = foxes
        self.sheep = sheep
        self.boats = boats
        if state is None:
            # each number represents how many of that item is on the wrong side of the river
            self.state = np.array([self.foxes, self.sheep, self.boats])
        else:
            self.state = state
        # both used to reconstruct the final path
        self.parent = parent
        self.action = action
        
    def is_goal_state(self):
        return np.array_equal(self.state, [0, 0, 0])
    
    def possible_moves(self):
        moves = []
        actions = []
        # how many boats move right
        for boats in range(1, self.state[2] + 1):
            for i in range(boats, 2 * boats + 1):
                # number of foxes and sheep that move
                for foxes in range(i+1):
                    for sheep in range(i+1):
                        if (foxes + sheep) == i and foxes <= self.state[0] and sheep <= self.state[1]:
                            actions.append(np.array([foxes, sheep, boats]))
                
        # how many boats move left
        for boats in range(1,self.boats - self.state[2] + 1):
            for i in range(boats, 2 * boats + 1):
                # number of foxes and sheep that move
                for foxes in range(i+1):
                    for sheep in range(i+1):
                        if (foxes + sheep) == i and foxes <= (self.foxes - self.state[0]) and sheep <= (self.sheep - self.state[1]):
                            actions.append(np.array([-foxes, -sheep, -boats])) # negative since they are moving to the wrong side
        
        # then check if its a valid move
        for i in actions:
            image = self.state - i
            if ((image[1] == 0) or (image[1] >= image[0])) and ((image[1] == self.sheep) or ((self.sheep - image[1]) >= (self.foxes - image[0]))):
                moves.append(i)
        return moves
    
    def get_child_state(self, move):
        new_state = self.state - move
        return RiverState(self.foxes, self.sheep, self.boats, new_state, self, move)
    
    def get_state(self):
        return self.state
    
    def get_parent(self):
        return self.parent
    
    def get_action(self):
        return self.action
    
    def __str__(self):
        line1 = ("#" * (self.foxes - self.state[0])) + ("f" * self.state[0]) + "|" + ("b" * self.state[2]) + ("~" * (self.boats - self.state[2] + 2)) + "|" + ("f" * (self.foxes - self.state[0])) + ("#" * self.state[0])
        line2 = ("#" * (self.sheep - self.state[1])) + ("s" * self.state[1]) + "|" + ("~" * (self.state[2] + 2)) + ("b" * (self.boats - self.state[2])) + "|" + ("s" * (self.sheep - self.state[1])) + ("#" * self.state[1])
        return line1 + "\n" + line2 + "\n"

    def __eq__(self, new):
        return np.array_equal(self.state, new.get_state())
    
class Game:
    def __init__(self):
        self.initial_state = RiverState(foxes = 5, sheep = 5, boats = 2)
        self.breadth_first_search(self.initial_state)
    
    def breadth_first_search(self, current_state):
        frontier = [current_state]
        explored = []
        
        while not current_state.is_goal_state() and len(frontier) > 0:
            current_state = frontier.pop(0)
            explored.append(current_state)
            
            moves = current_state.possible_moves()
            for move in moves:
                new_state = current_state.get_child_state(move)
                if new_state not in frontier and new_state not in explored:
                    frontier.append(new_state)
        
        if current_state.is_goal_state():
            print("Soultion Found!\n")
            
            solution = []
            while current_state.get_parent() is not None:
                solution.append(current_state)
                current_state = current_state.get_parent()
            solution.append(current_state)
            
            for state in solution[::-1]:
                action = state.get_action()
                if action is not None:
                    print("Move over %d boats holding %d foxes and %d sheep." % (action[2], action[0], action[1]))
                else:
                    print("Starting River:")
                print(state)
        else:
            print("No Solution")

game = Game()
            

Soultion Found!

Starting River:
fffff|bb~~|#####
sssss|~~~~|#####

Move over 2 boats holding 2 foxes and 2 sheep.
##fff|~~~~|ff###
##sss|~~bb|ss###

Move over -2 boats holding 0 foxes and -2 sheep.
##fff|bb~~|ff###
sssss|~~~~|#####

Move over 2 boats holding 1 foxes and 3 sheep.
###ff|~~~~|fff##
###ss|~~bb|sss##

Move over -2 boats holding -1 foxes and -1 sheep.
##fff|bb~~|ff###
##sss|~~~~|ss###

Move over 2 boats holding 1 foxes and 3 sheep.
###ff|~~~~|fff##
#####|~~bb|sssss

Move over -2 boats holding 0 foxes and -2 sheep.
###ff|bb~~|fff##
###ss|~~~~|sss##

Move over 2 boats holding 2 foxes and 2 sheep.
#####|~~~~|fffff
#####|~~bb|sssss



If you use the provided template you could then try to repduce the following printed output.

In [2]:
g = Game()
goal_node = g.breadth_first_search()
print("The goal node is", goal_node)

Soultion Found!

Starting River:
fff|b~~|###
sss|~~~|###

Move over 1 boats holding 1 foxes and 1 sheep.
#ff|~~~|f##
#ss|~~b|s##

Move over -1 boats holding 0 foxes and -1 sheep.
#ff|b~~|f##
sss|~~~|###

Move over 1 boats holding 2 foxes and 0 sheep.
###|~~~|fff
sss|~~b|###

Move over -1 boats holding -1 foxes and 0 sheep.
##f|b~~|ff#
sss|~~~|###

Move over 1 boats holding 0 foxes and 2 sheep.
##f|~~~|ff#
##s|~~b|ss#

Move over -1 boats holding -1 foxes and -1 sheep.
#ff|b~~|f##
#ss|~~~|s##

Move over 1 boats holding 0 foxes and 2 sheep.
#ff|~~~|f##
###|~~b|sss

Move over -1 boats holding -1 foxes and 0 sheep.
fff|b~~|###
###|~~~|sss

Move over 1 boats holding 2 foxes and 0 sheep.
##f|~~~|ff#
###|~~b|sss

Move over -1 boats holding 0 foxes and -1 sheep.
##f|b~~|ff#
##s|~~~|ss#

Move over 1 boats holding 1 foxes and 1 sheep.
###|~~~|fff
###|~~b|sss



TypeError: breadth_first_search() missing 1 required positional argument: 'current_state'

### Solution
The solution to this exercise will be made available through the course page once you submit your version. If you are completely stuck you can submit the file unfinished to see the solution – the submission is not graded, but you should try to get your version working.

### Advanced Extension
In the Tower of Hanoi example you implemented *depth first* search, which is similar to breadth first search. Thinking about the trade-offs between each algorithm, do you think depth first search would be effective for the Missionaries and Cannibals problem?

There is another technique called *iterative deepening* which combines the benefits of both algorithms. The basic idea is to repeatedly try a limited version of depth first *tree search* to ever increasing depths. First of all we try a depth first search limited to depth 0: this will simply be the root note. Then we try depth first search to a limit of 1, which is just the children of the root node. Then a limit of 2, and so on. We stop when we find the goal, or if we do not find any nodes with the given depth (in other words we have explored the entire graph). Because we are using the *tree* version of the algorithm (even on a graph problem), we do not need to store the `explored` set, and the memory requirements are far less.

You can read more about this technique in section 3.4.5 of Russell and Norvig – the section is on page 88 but I would recommend you start reading from page 86 which explains the limitations of depth-first search, the motivation for depth-limited search, and the natural conclusion: iterative deepening. 

This technique provides a excellent trade-off in terms of memory and time efficiency. It may seem wasteful to generate the early states multiple times, but there are not many states in the early part of the search tree. This means that iterative deepening is roughly the same as breadth first search for total computational time and has much better memory performance.

Here is the pseudocode from Russell and Norvig:

<br>
<center>
<img src="resources/iterative-deepening.png" width=60%>
</center>

**Task:** Try writing an iterative deepening solution to either the Missionaries and Cannibals problem or the Tower of Hanoi problem. Or, you could try another search technique from the textbook. Share your results on the forum!